In [1]:
import pandas as pd
import numpy as np
import psycopg2
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import LabelEncoder
import dotenv

dotenv.load_dotenv()

PG_PASSWORD = os.getenv("PG_PASSWORD")
PG_USER = os.getenv("PG_USER")
PG_HOST = os.getenv("PG_HOST")
PG_PORT = os.getenv("PG_PORT")
PG_DATABASE = os.getenv("PG_DB")

# Verbinding maken met de database
conn = psycopg2.connect(
    host=PG_HOST,
    database=PG_DATABASE,
    user=PG_USER,
    password=PG_PASSWORD,
    port=PG_PORT,
    sslmode="require",
)

# Haal de data op met de gegeven query
query = """
WITH transition_data AS (
    SELECT
        spadl.game_id,
        spadl.period_id,
        spadl.player_id,
        spadl.team_id,
        spadl.start_x,
        spadl.start_y,
        spadl.end_x,
        spadl.end_y,
        spadl.seconds,
        spadl.action_type,
        spadl.result,
        spadl.bodypart,
        m.home_score,
        m.away_score,
        CASE
            WHEN spadl.team_id = m.home_team_id THEN m.home_score
            ELSE m.away_score
        END AS score_team,

        CASE
            WHEN spadl.team_id = m.home_team_id THEN m.away_score
            ELSE m.home_score
        END AS score_opponent,

        SQRT(POWER(spadl.end_x - spadl.start_x, 2) + POWER(spadl.end_y - spadl.start_y, 2)) / NULLIF(spadl.seconds, 0) AS transition_speed,
        t1.team_name AS team_name,
        t2.team_name AS opponent_team_name,
        me.ball_owning_team  -- Add the ball_owning_team column from matchevents table
    FROM
        spadl_actions spadl
    JOIN teams t1 ON spadl.team_id = t1.team_id
    JOIN teams t2 ON spadl.team_id != t2.team_id
    JOIN matches m ON spadl.game_id = m.match_id
    JOIN matchevents me ON spadl.game_id = me.match_id
    WHERE
        spadl.action_type IN ('9', '10', '18')  -- Filter for defensive actions (tackle, interception, clearance)
)
SELECT
    game_id,
    period_id,
    player_id,
    team_id,
    team_name,
    opponent_team_name,
    start_x,
    start_y,
    end_x,
    end_y,
    seconds,
    action_type,
    result,
    transition_speed,
    score_team,
    score_opponent,
    ball_owning_team,  -- Include ball_owning_team to track possession
    CASE
        WHEN transition_speed < 5 THEN 'fast'
        ELSE 'slow'
    END AS transition_type
FROM
    transition_data
WHERE
    ball_owning_team = transition_data.team_id  -- Check if the ball is now owned by your team after the defensive action
ORDER BY
    game_id, period_id, seconds
LIMIT 10000;

"""


data = pd.read_sql(query, conn)


conn.close()


print(data.head())


C:\Users\fleur\AppData\Local\Temp\ipykernel_28036\2407409203.py:105: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, conn)


                     game_id  period_id                 player_id  \
0  5ow2wa823rjft38oh48b4ror8          1  8lqo2ajhgjockasmd198nai1   
1  5ow2wa823rjft38oh48b4ror8          1  8lqo2ajhgjockasmd198nai1   
2  5ow2wa823rjft38oh48b4ror8          1  8lqo2ajhgjockasmd198nai1   
3  5ow2wa823rjft38oh48b4ror8          1  8lqo2ajhgjockasmd198nai1   
4  5ow2wa823rjft38oh48b4ror8          1  8lqo2ajhgjockasmd198nai1   

                     team_id team_name  opponent_team_name  start_x  start_y  \
0  4dtif7outbuivua8umbwegoo5    Dender         Club Brugge    38.64   64.396   
1  4dtif7outbuivua8umbwegoo5    Dender            Mechelen    38.64   64.396   
2  4dtif7outbuivua8umbwegoo5    Dender  Sporting Charleroi    38.64   64.396   
3  4dtif7outbuivua8umbwegoo5    Dender          Anderlecht    38.64   64.396   
4  4dtif7outbuivua8umbwegoo5    Dender            Kortrijk    38.64   64.396   

    end_x  end_y  seconds action_type result  transition_speed  score_team  \
0  53.865   68.0     61.0 

In [2]:


X = data.drop(['transition_type'], axis=1)


y = data['transition_type'].apply(lambda x: 1 if x == 'fast' else 0)


label_encoder = LabelEncoder()


X['team_name'] = label_encoder.fit_transform(X['team_name'])
X['opponent_team_name'] = label_encoder.fit_transform(X['opponent_team_name'])
X['game_id'] = label_encoder.fit_transform(X['game_id'])
X['player_id'] = label_encoder.fit_transform(X['player_id'])
X['action_type'] = label_encoder.fit_transform(X['action_type'])
X['ball_owning_team'] = label_encoder.fit_transform(X['ball_owning_team'])
X['team_id'] = label_encoder.fit_transform(X['team_id'])







In [3]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:

from sklearn.model_selection import cross_val_score
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
cv_scores = cross_val_score(rf_classifier, X, y, cv=5)
rf_classifier.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [5]:

y_pred = rf_classifier.predict(X_test)

In [6]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00      2000

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000

